In [2]:
import numpy as np
from ase.io import read, write
from ase.visualize import view
from ase.data.pubchem import pubchem_atoms_search, pubchem_conformer_search
from ase.build import add_adsorbate
from ase.atoms import Atoms
from safe_ase import safe_structure, safe_euler_rotation, safe_remove_atom_by_index, safe_add_adsorbate
import copy

In [3]:
mono = read('mono4.cif')
mono2 = read('mono4.cif')

In [50]:
view(mono)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [40]:
mono2.positions += [0,0,6]
# mono2.cell += [0,0,12]

In [28]:
view(mono + mono2)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [41]:
layered = mono + mono2

In [49]:
view(layered*(2,2,1))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [22]:
mono2.cell == mono.cell

array([[ True,  True, False],
       [ True,  True, False],
       [ True,  True, False]])

In [46]:
layered.cell[2,2]

22.47

In [47]:
layered.cell[2,2] = layered.positions[:,2].max() + 5

In [25]:
def build_two_layers(distance):
    intra = 2.137 # Guarda el valor de la diferencia en la estructura de maple
    down_layer = read('mono4.cif')
    up_layer = read('mono4.cif')
    up_layer.positions += [0, 0, intra + distance]
    both_layers = down_layer + up_layer
    both_layers.cell[2,2] = both_layers.positions[:,2].max() + 5
    return both_layers

In [58]:
view(build_two_layers(6)*(2,2,1))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [5]:
def freeze_atoms(structure: Atoms, freeze_index = []) -> np.array:
    cons = np.zeros((len(structure),3), dtype=int)
    for f in freeze_index:
        cons[f] = 1
    return cons

In [6]:
def pretty_coordinates(structure: Atoms, frozen_atoms = []):
    symbols = np.array(structure.get_chemical_symbols())
    symbols.shape = (len(symbols), 1) # Fix dimensions in order to concatenate
    matched = np.concatenate((symbols, structure.get_positions(), freeze_atoms(structure, frozen_atoms)), axis=1)
    # matched = np.concatenate((symbols, structure.get_positions()), axis=1)
    pretty = ['      '.join(coord) for coord in matched]
    return pretty

In [7]:
def pretty_cell(structure: Atoms, distance: float):
    cell = structure.cell
    cell[2] = cell[2] + [0,0,distance]
    axis = [ c for c in cell]
    return [ '      '.join([str(i) for i in a]) for a in axis] # Doble recursion

In [8]:
from jinja2 import Environment, FileSystemLoader

In [9]:
environment = Environment(loader=FileSystemLoader("templates/"))

In [ ]:
input_render = {
    "prefijo": "twosubs",
    "dirSalida": 'salida',
    "nat": len(second_sub),
    "coordenadas": pretty_coordinates(second_sub),
    "ejes": pretty_cell(second_sub,0)
}

In [10]:
def gen_two_layers_input(template_name: str,
    prefix: str, distance: float,
    frozen_atoms = []
):
    environment = Environment(loader=FileSystemLoader("templates/"))
    template = environment.get_template(template_name)
    both = build_two_layers(distance)
    s_distance = "{:.2f}".format(distance)
    input_render = {
    "prefijo": prefix + s_distance,
    "dirSalida": prefix + s_distance,
    "nat": len(both),
    "coordenadas": pretty_coordinates(both, frozen_atoms),
    "ejes": pretty_cell(both, distance)
    }
    with open(prefix + s_distance + "-" + template_name,'w',encoding = 'utf-8') as f:
        f.write(template.render(input_render))
        f.close()

In [21]:
[gen_two_layers_input('scf.in', 'exfoliation', i) for i in np.linspace(3.5,4.5, 25)]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [16]:
np.linspace(4.0,4.5,20)

array([4.        , 4.02631579, 4.05263158, 4.07894737, 4.10526316,
       4.13157895, 4.15789474, 4.18421053, 4.21052632, 4.23684211,
       4.26315789, 4.28947368, 4.31578947, 4.34210526, 4.36842105,
       4.39473684, 4.42105263, 4.44736842, 4.47368421, 4.5       ])

In [20]:
gen_two_layers_input('scf.in', 'exfoliation', 8)

In [28]:
view(build_two_layers(4.16))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>